In [1]:
import pandas as pd
import re
import gensim.corpora as corpora
import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import spacy
from pprint import pprint
from nltk.stem.snowball import SnowballStemmer

In [2]:
df_file = pd.read_csv('cleaning_contenido2.csv')
##df_file = pd.read_csv('base_total.csv',sep='|')

In [3]:
#LIMPIEZA
df_corpus= df_file[['contenido']].copy()
df_corpus=df_corpus[1:]
df_corpus=df_corpus.dropna()
df_corpus['contenido'] = df_corpus['contenido'].map(lambda x: re.sub('\s+', ' ', x))
df_corpus['contenido'] = df_corpus['contenido'].map(lambda x: re.sub('[_,";¿*&()=:+#\|@.!?]', ' ', x))
df_corpus['contenido'] = df_corpus['contenido'].map(lambda x: x.lower())
palabras_cortas = re.compile(r'\W*\b\w{1,3}\b')
df_corpus['contenido'] = df_corpus['contenido'].map(lambda x: palabras_cortas.sub('', x))

df_corpus.head()


,contenido
1,coronavirus este lunes abril ministro salud j...
2,ministerio salud minsal informó mañana este l...
3,coronavirus como resultado acuerdos mesa soci...
4,casos coronavirus chile confirmados total 6927...
5,casos covid chile tiene nexo trazable encuent...


In [4]:
#Tokenizacion
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 

In [5]:
datos= df_corpus.values.tolist()
datos_palabras= list(sent_to_words(datos))


In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('spanish')
newStopWords = ['coronavirus','chile','casos','detalles','agrego','embargo','forma','semanas','fecha','francia','espana','cada','agregó','parte','caso','dias','respecto','momento','covid','ascom','tikitakas','personas','tipo','tenía','jorge','habrá','habra','medios','perú','casa','cuales',
                'marzo','abril','lunes','semana','despues','hora','también','tambien','martes','radio','lugar','lugares','anuncio','anunciar',
                'miércoles','jueves','viernes','sábado','domingo','enero','comentarios','reservamos','febrero','italia','según','segun','horas','aquí','virus','veces','radio','eliodoro', 'yanez', 'providencia',
                 'reservamos','derecho','bloquear','usuarios','agresivos','commons','noticias','indique','contrario','eliminar','comentarios','debate','espacio','wordpress','desarrollado','detallo','estime','conveniente','manteniendo',]
stop_words.extend(newStopWords)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jmerc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out
def to_array(texts):
    arreglo_v=[]
    for docv in range(len(texts)):
        for wordv in data_lemmatized[docv].split():
            if wordv is None:
                pass
            else:
                arreglo_v.append(wordv)
    return arreglo_v
def lemma_sustantivos(texts):
    text_out = []
    for sent in texts:
        doc = nlp1(" ".join(sent)) 
        text_out.append(" ".join([token.text for token in doc.noun_chunks])) 
    return text_out
        
        

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(datos_palabras,min_count=5) # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)

data_words_nostops = remove_stopwords(datos_palabras)
data_words_bgrams =  make_bigrams(data_words_nostops)
nlp1 = spacy.load("es_core_news_sm")
data_lemmatized =  lemma_sustantivos(data_words_bgrams)

dataset = [d.split() for d in data_lemmatized]
id2word = corpora.Dictionary(dataset)
# Create Corpus
texts = dataset
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]







In [9]:
dataset = [d.split() for d in data_lemmatized]
id2word = corpora.Dictionary(dataset)
# Create Corpus
texts = dataset
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=15, 
                                       random_state=100,
                                       chunksize=50,
                                       passes=10,
                                       per_word_topics=True)



In [12]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14     0.066869  0.012190       1        1  32.266460
9      0.108447  0.053699       2        1  12.133153
12     0.115778  0.005484       3        1  11.950552
8      0.166738 -0.049687       4        1   9.914553
2      0.078663 -0.032488       5        1   9.625396
10     0.089482  0.003978       6        1   5.759247
3     -0.065412  0.198623       7        1   5.214843
11    -0.074380  0.019041       8        1   4.222734
1      0.001676 -0.011404       9        1   3.508496
7     -0.100643 -0.100633      10        1   1.905861
6     -0.051079 -0.007145      11        1   1.197022
4     -0.087622 -0.010152      12        1   0.920194
13    -0.065128 -0.030728      13        1   0.588785
5     -0.106503 -0.011843      14        1   0.521256
0     -0.076888 -0.038934      15        1   0.271449, topic_info=           Term        Freq       Total Category  logprob  loglift
639    millones   37.000000   37.000000  Default  30.0000  30.0000
18     pandemia  112.000000  112.000000  Default  29.0000  29.0000
33    pacientes   59.000000   59.000000  Default  28.0000  28.0000
17         pais  123.000000  123.000000  Default  27.0000  27.0000
275     medidas  154.000000  154.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
2061    cuidado    0.014818    5.408216  Topic15  -7.9055   0.0093
348       salud    0.021386  129.706940  Topic15  -7.5386  -2.8011
69     gobierno    0.021337  161.160492  Topic15  -7.5410  -3.0206
609      crisis    0.020138  129.058823  Topic15  -7.5988  -2.8563
18     pandemia    0.018159  112.704361  Topic15  -7.7022  -2.8242

[1007 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2563      8  0.942583  abordaje
870       1  0.544592  acciones
870       3  0.090765  acciones
870       4  0.045383  acciones
870       6  0.090765  acciones
...     ...       ...       ...
387       6  0.058892     zonas
387       7  0.058892     zonas
387       8  0.058892     zonas
387       9  0.058892     zonas
387      11  0.058892     zonas

[1894 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 10, 13, 9, 3, 11, 4, 12, 2, 8, 7, 5, 14, 6, 1])

In [14]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

AttributeError: 'LdaMulticore' object has no attribute 'components_'

In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords